In [11]:
import os, sys
import importlib
import logging
import tempfile
from typing import *

import tqdm

import numpy as np
import pandas as pd
from sklearn import metrics

import torch
from torch.utils.data import Subset
import skorch
import skorch.helper

SRC_DIR = os.path.join(os.path.dirname(os.getcwd()), "tcr")
assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
sys.path.append(SRC_DIR)
import data_loader as dl
import utils
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)
import conv

FILT_EDIT_DIST = False

DEVICE = utils.get_device(3)
TRAINED_MODEL_DIR = "/home/wukevin/projects/tcr/tcr_models"

In [3]:
# Load in the PIRD dataset
pird_data = dl.load_pird(with_antigen_only=True)
pird_data = pird_data.loc[~pd.isnull(pird_data['CDR3.beta.aa'])]
pird_data.head()

INFO:root:PIRD data 0.1655 data labelled with antigen sequence
INFO:root:PIRD: Removing 95 entires with non amino acid residues
INFO:root:Entries with antigen sequence: 8429/51044
INFO:root:Unique antigen sequences: 73
INFO:root:PIRD data TRA/TRB instances: Counter({'TRB': 46428, 'TRA': 4011, 'TRA-TRB': 605})
INFO:root:PIRD entries with TRB sequence: 4607
INFO:root:PIRD entries with TRB sequence: 47040
INFO:root:PIRD entries with TRA and TRB:  605


,ICDname,Disease.name,Category,Antigen,Antigen.sequence,HLA,Locus,CDR3.alpha.aa,CDR3.beta.aa,CDR3.alpha.nt,...,Cell.subtype,Prepare.method,Evaluate.method,Case.num,Control.type,Control.num,Filteration,Journal,Pubmed.id,Grade
0,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIEHTNSGGSNYKLTF,CASSLEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVHTNSGGSNYKLTF,CASSPEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
2,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVKTNSGGSNYKLTF,CASSFEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1845,A15,Tuberculosis,Pathogen,Rv1195,ADTLQSIGATTVASN,DRB1*15:03,TRA-TRB,CAGAGGGGFKTIF,CASSVALASGANVLTF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1846,A15,Tuberculosis,Pathogen,Rv1195,ADTLQSIGATTVASN,DRB1*15:03,TRA-TRB,CAGPTGGSYIPTF,CASSVALATGEQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5


In [13]:
tcrdb_data = dl.load_tcrdb()
tcrdb_data = tcrdb_data.loc[tcrdb_data['tra_trb'] == 'TRB']
tcrdb_data.head()

,accession,RunId,AASeq,cloneFraction,tra_trb
0,PRJNA330606,SRR4102112,CANTGTGFNEQFF,0.008305,TRB
1,PRJNA330606,SRR4102112,CASSHTRGVGTQYF,0.003841,TRB
2,PRJNA330606,SRR4102112,CSGVHEQYF,0.003824,TRB
3,PRJNA330606,SRR4102112,CASSLPNGEGSSYEQYF,0.002825,TRB
4,PRJNA330606,SRR4102112,CASSQGGIAGDVYEQYF,0.002614,TRB


In [1]:
importlib.reload(dl)

REDUCE_LR_ON_PLATEAU_PARAMS = {
    "mode": "min",
    "factor": 0.1,
    "patience": 10,
    "min_lr": 1e-6,
}

tcrdb_neg_rng = np.random.default_rng(seed=64)

def convnet_on_antigen(antigen:str) -> Tuple[str, float]:
    """
    Train a convnet on the antigen and return Tuple[antigen, test_auprc]
    If invalid, return nan
    """
    pird_pos_table = pird_data.loc[pird_data['Antigen.sequence'] == antigen]
    pird_pos_trbs = list(pird_pos_table['CDR3.beta.aa'])
    if len(pird_pos_trbs) < 20:
        return antigen, np.nan
    
    # Get a negative set of sequences from TCRdb, sampled at 5 negatives per positive sequence
    tcrdb_trbs = tcrdb_data['AASeq']
    rand_neg_trbs = list(tcrdb_data.iloc[tcrdb_neg_rng.choice(
        np.arange(len(tcrdb_trbs)), size=int(len(pird_pos_trbs) * 5), replace=False
    )]['AASeq'])
    
    full_dset = dl.TCRSupervisedIdxDataset(
        pird_pos_trbs + rand_neg_trbs,
        np.array([1] * len(pird_pos_trbs) + [0] * len(rand_neg_trbs))
    )
    train_dset_raw = dl.DatasetSplit(full_dset, split='train', valid=0.0, test=0.3)
    test_dset = dl.DatasetSplit(full_dset, split='test', valid=0.0, test=0.3)

    # Filter out training sequences too similar to test sequences
    if FILT_EDIT_DIST:
        train_seqs = train_dset_raw.all_sequences()
        test_seqs = test_dset.all_sequences()
        train_dists = dl.min_dist_train_test_seqs(train_seqs, test_seqs)
        far_idx = np.where(train_dists >= 2)[0]  # Keep only train seqs at least 2 edits away
        train_dset = Subset(train_dset_raw, far_idx)
    else:
        train_dset = train_dset_raw
    
    torch.manual_seed(42)
    with tempfile.TemporaryDirectory() as tmpdir:
        logging.info(f"Running in temporary dir: {tmpdir}")
        net = skorch.NeuralNet(
            module=conv.OnePartConvNet,
            module__use_embedding=True,
            module__n_output=2,
            module__max_input_len=full_dset.max_len,
            criterion=torch.nn.CrossEntropyLoss,
            optimizer=torch.optim.Adam,
            max_epochs=250,
            batch_size=512,
            lr=1e-3,
            callbacks=[
                skorch.callbacks.LRScheduler(
                    policy=torch.optim.lr_scheduler.ReduceLROnPlateau,
                    mode='min',
                    factor=0.1,
                    patience=10,
                    min_lr=1e-6,
                ),
                skorch.callbacks.GradientNormClipping(gradient_clip_value=5),
                skorch.callbacks.EpochScoring(
                    "average_precision",
                    lower_is_better=False,
                    on_train=False,
                    name="valid_auprc",
                ),
                skorch.callbacks.EarlyStopping(
                    patience=25,
                    monitor="valid_auprc",
                    lower_is_better=False,
                ),
                skorch.callbacks.Checkpoint(  # Seems to cause errors if placed before scoring
                    dirname=tmpdir,
                    fn_prefix="net_",
                    monitor="valid_auprc_best",
                ),
            ],
            device=3,
        )
        net.classes_ = np.unique(test_dset.all_labels())
        net.fit(train_dset)
        # Restore best weights and evaluate on test set
        cp = skorch.callbacks.Checkpoint(dirname=tmpdir, fn_prefix="net_")
        # net.load_params(checkpoint=cp)  # Performance is better with this disabled
        test_truth = test_dset.all_labels()
        test_preds = net.predict_proba(test_dset)[:, 1]
        test_auprc = metrics.average_precision_score(test_truth, test_preds)
    return antigen, test_auprc
    
convnet_on_antigen("FPRPWLHGL")

IndentationError: expected an indented block (<ipython-input-1-e93a52739a6c>, line 44)

In [17]:
antigen_auprc_pairs = [convnet_on_antigen(a) for a in utils.dedup(pird_data['Antigen.sequence'])]
len(antigen_auprc_pairs)

INFO:root:Max len not set, using empirical max len of 26
INFO:root:Using maximum length of 26
INFO:root:Split train with 970 examples
INFO:root:Split test with 416 examples
INFO:root:Running in temporary dir: /tmp/tmprzvpv1m9


  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        0.5475         0.1221        0.6887     +  0.0244
      2        0.3900         0.2035        0.6639     +  0.0539
      3        0.3453         0.3061        0.6462     +  0.0409
      4        0.3056         0.3584        0.6313     +  0.0460
      5        0.2774         0.3771        0.6102     +  0.0288
      6        0.2589         0.3948        0.5801     +  0.0245
      7        0.2408         0.4083        0.5403     +  0.0241
      8        0.2228         0.4139        0.4951     +  0.0242
      9        0.2085         0.4256        0.4526     +  0.0244
     10        0.1964         0.4208        0.4193        0.0242
     11        0.1834         0.4226        0.3955        0.0241
     12        0.1698         0.4205        0.3770        0.0240
     13        0.1578         0.4288        0.3600     +  0.0240
     14        0.1473    

INFO:root:Max len not set, using empirical max len of 20
INFO:root:Using maximum length of 20
INFO:root:Split train with 239 examples
INFO:root:Split test with 103 examples
INFO:root:Running in temporary dir: /tmp/tmp4m067d32


  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        1.2464         0.1333        0.7201     +  0.0179
      2        0.8146         0.1268        0.6895        0.0114
      3        0.5351         0.1169        0.6591        0.0206
      4        0.3710         0.1233        0.6306        0.0139
      5        0.2826         0.1233        0.6011        0.0117
      6        0.2344         0.1417        0.5706     +  0.0182
      7        0.2041         0.1726        0.5392     +  0.0116
      8        0.1820         0.1500        0.5081        0.0093
      9        0.1633         0.1333        0.4773        0.0087
     10        0.1460         0.1339        0.4471        0.0088
     11        0.1297         0.1394        0.4178        0.0088
     12        0.1142         0.1736        0.3900     +  0.0087
     13        0.0997         0.2111        0.3638     +  0.0088
     14        0.0867    

INFO:root:Max len not set, using empirical max len of 20
INFO:root:Using maximum length of 20
INFO:root:Split train with 239 examples
INFO:root:Split test with 103 examples
INFO:root:Running in temporary dir: /tmp/tmp81bsk6rt


...

/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


     21        0.0191            nan        0.2283        0.0163
     22        0.0174            nan        0.2130        0.0139
     23        0.0160            nan        0.1990        0.0124
     24        0.0147            nan        0.1865        0.0102
Stopping since valid_auprc has not improved in the last 25 epochs.


/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/wukevin/miniconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
INFO:root:Max len not set, using empirical max len of 20
INFO:root:Using maximum length of 20
I

  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        1.1910         0.0972        0.6904     +  0.0530
      2        0.6289         0.1085        0.6359     +  0.0259
      3        0.4077         0.1159        0.5790     +  0.0226
      4        0.3352         0.1189        0.5210     +  0.0218
      5        0.3094         0.1261        0.4664     +  0.0347
      6        0.2943         0.1351        0.4189     +  0.0328
      7        0.2798         0.1444        0.3806     +  0.0222
      8        0.2637         0.1696        0.3514     +  0.0880
      9        0.2463         0.1824        0.3299     +  0.0229
     10        0.2290         0.1884        0.3144     +  0.0227
     11        0.2131         0.1933        0.3033     +  0.0359
     12        0.1991         0.2022        0.2951     +  0.0355
     13        0.1872         0.2136        0.2891     +  0.0229
     14        0.1766    

73

In [18]:
antigen_auprc_pairs = [pair for pair in antigen_auprc_pairs if not np.isnan(pair[1])]
len(antigen_auprc_pairs)

26

In [19]:
df = pd.DataFrame(
    [pair[1] for pair in antigen_auprc_pairs],
    index=[pair[0] for pair in antigen_auprc_pairs],
    columns=['ConvNet']
)
df

,ConvNet
LLWNGPMAV,0.530086
RPRGEVRFL,0.386815
ATDALMTGY,0.878661
HSKKKCDEL,0.268149
KAFSPEVIPMF,0.561982
KRWIILGLNK,0.413079
TPQDLNTML,0.388053
EIYKRWII,0.162382
HPKVSSEVHI,0.583317
IIKDYGKQM,0.623517


In [20]:
if FILT_EDIT_DIST:
    df.to_csv("antigen_cv_convnet_baseline_edit_dist_filt.csv")
else:
    df.to_csv("antigen_cv_convnet_baseline.csv")